First we will initialize our filiBERTo tokenizer like before, and use it to encode our data.

In [1]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
import torch
from pathlib import Path
from tqdm.auto import tqdm
import random

In [2]:
# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = ByteLevelBPETokenizer(
    './filiberto/filiberto-vocab.json',
    './filiberto/filiberto-merges.txt'
)
# set [CLS] and [SEP] to be added to start-end of sequences
tokenizer._tokenizer.post_processor = BertProcessing(
    ('[SEP]', tokenizer.token_to_id('[SEP]')),
    ('[CLS]', tokenizer.token_to_id('[CLS]'))
)
# truncate anything more than 512 tokens in length
tokenizer.enable_truncation(max_length=512)
# and enable padding to 512 too
tokenizer.enable_padding(length=512, pad_token='[PAD]')

In [3]:
def mlm(tensor):
    # create random array of floats with equal dims to tensor
    rand = torch.rand(tensor.shape)
    # mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
    mask_arr = (rand < .15) * (tensor != 0) * (tensor != 1) * (tensor != 2)
    # loop through each row in tensor (cannot do in parallel)
    for i in range(tensor.shape[0]):
        # get indices of mask positions from mask array
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        # mask tensor
        tensor[i, selection] = 3
    return tensor

In [4]:
paths = [str(x) for x in Path('../../data/text/oscar_it').glob('**/*.txt')]
# initialize lists of tensors
input_ids = []
mask = []
labels = []
# open all files, encode and add to single dataset
for path in tqdm(paths[:50]):
    # open the file and split into list by newline characters
    with open(path, 'r', encoding='utf-8') as fp:
        lines = fp.read().split('\n')
    # encode
    sample = tokenizer.encode_batch(lines)
    # convert tokens to tensor
    labels.append(torch.tensor([x.ids for x in sample]))
    # create attention mask tensor
    mask.append(torch.tensor([x.attention_mask for x in sample]))
    # mask ~15% of tokens to create inputs
    input_ids.append(mlm(labels[-1].detach().clone()))
# convert lists of tensors into tensors
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)
    

100%|██████████| 20/20 [01:49<00:00,  5.49s/it]


We have 10000 tokenized sequences, each containing 512 tokens:

In [5]:
input_ids.shape

torch.Size([200000, 512])

We can see the special tokens here, `1` is our **\[CLS\]** token, `2` our **\[SEP\]** token, `3` our **\[MASK\]** token, and at the end we have two `0` - or **\[PAD\]** - tokens:

In [6]:
input_ids[0]

tensor([    1,   693, 18623,  1358,  7752,   292,  1056,   280,  7405,  6321,
          776,   726,  2145,   280,    11, 10205,  3778,  1266,  1810,     3,
            3,  1142, 10293,    30,     3,   267,  1340,    16,   385,  3375,
          458,     3,  5942,   376, 25475,  2870,  1201,   391,  2691,   421,
        17927, 16996,   739,   305,   306, 22814,     3,  7950, 17824,   980,
          435, 18388,  1475,   275,  2597,   391,    37, 24909,   739,  2689,
        27869,   275,  5803,   625,   770, 13459,   483,     3,   275, 12870,
          532,    18,   680,  3867, 24138,   376,  7752, 17630, 18623,  1134,
         8882,   269,   431,   287,     3,   483,  8041,     3,   275,  5286,
           18,     3,   367,   275,  6161,     3, 10528,   570,  1594, 13181,
           18,   458,    16,     3,   456,  2150, 12054,    16,   500,   317,
         6122,     3,  3329,   570,  1594, 13181,   280, 14634,    18,   763,
        12656,     3,  2484,  6544,     3,     3,  9106,     3, 

In [7]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}

In [8]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

Next we initialize our `Dataset`.

In [9]:
dataset = Dataset(encodings)

And initialize the dataloader, which will load the data into the model during training.

In [10]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

And move onto building our model, we first need to create a BERT config object, which will describe which features we want to initialize our BERT model with.

In [11]:
from transformers import BertConfig

config = BertConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab set in previous notebook
    hidden_size=768,
    num_hidden_layers=12,
    pad_token_id=0
)

Then we import and initialize a BERT model with a language modeling head.

In [12]:
from transformers import BertLMHeadModel

model = BertLMHeadModel(config)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


And now we move onto training. First we setup GPU/CPU usage.

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

Activate the training mode of our model, and initialize our optimizer (Adam with weighted decay - reduces chance of overfitting).

In [15]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [18]:
writer = torch.utils.tensorboard.SummaryWriter()

Now we move onto the training loop.

In [20]:
from tqdm import tqdm  # for our progress bar

epochs = 2
step = 0

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # take loss for tensorboard
        writer.add_scalar('Loss/train', loss, step)
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
        step += 1

Epoch 1: 100%|██████████| 12500/12500 [1:22:20<00:00,  2.53it/s, loss=0.31]


In [21]:
model.save_pretrained('./filiberto')